# Using PiRecorder

<span style="font-size:larger;">This notebook will give a quick tutorial of how to use the PiRecorder module of the picamera package. For detailed documentation, go to [http://jollejolles.github.io/pirecorder](http://jollejolles.github.io/pirecorder)</span>

## Starting a PiRecorder instance

PiRecorder is the main module of the *pirecorder* package and facilitates the controlled and automated recording of images and videos. As it is a class instancem it needs to be stored as a variable.

The first time the PiRecorder instance is run, automatically a `pirecorder` directory will be created in the user's home directory with a default configuration file (`pirecorder.conf`) that will be used as the basis for future recordings. To initiate PiRecorder:

In [ ]:
import pirecorder
rec = pirecorder.PiRecorder()

I use the variable name `Rec`, but any variable is fine as long as you are consistent in using it.

## Using (custom) configuration files

As is explained in the [pirecorder package page](http://github.com/jollejolles/pirecorder/docs/pirecorder-package.md), one of the main features of PiRecorder is that it works with a simple-to-use configuration file to set all your camera and recording settings to easily run repeated and automated recordings.

The configuration file can be set with the `configfile` parameter when initiating the PiRecorder instance, which defaults to "pirecorder.conf". For example, to work with a special config file for infrared recordings, you can run:

In [ ]:
rec = pirecorder.PiRecorder(configfile = "irsettings.conf")

The configuration files can be simply changed manually with any text editor or with your favorite terminal editor (e.g. `nano pirecorder/pirecorder.conf`). You can also easily update the configuration via the `settings` function and only add the parameters you want to change. For example:

In [ ]:
rec.settings(label="test", rectype="img")

An overview of all the recording parameters with concise description that are possible to set is provided at the bottom of this page and can also be called directly in Python:

In [ ]:
print(rec.settings.__doc__)

To see your current configuration settings, simply type in:

In [ ]:
print(rec.config)

Here is an overview of all parameters you can set

In [ ]:
rec.settings(recdir = "recordings", label = "test", rectype = "img",  rotation = 0, brighttune = 0,   \
             gains = (1.0, 2.5), brightness = 45, contrast = 20, saturation = -100, iso = 200,        \
             sharpness = 50, compensation = 0, shutterspeed = 10000, imgdims = (3280,2464),           \
             viddims = (1640,1232), imgfps = 1, vidfps = 24, imgwait = 1.0, imgnr = 60, imgtime = 60, \
             imgquality = 50, vidduration = 10, viddelay = 0, vidquality = 11)

## Where and what to record

Two important configurations to set are the `recdir` and `rectype` parameters. By default PiRecorder will store all media in a recordings folder inside the pirecorder folder (`pirecorder/recordings`). You can change this to any folder name that you like. If no name is provided, the files will be directly stored in the users' home directory. If you want to store media on a mounted media, name the mounted folder "NAS" and use `recdir = "NAS"`, then it will automatically check if it is mounted and otherwise record in the default location.

With PiRecorder you can record single images (`rectype = img`), a sequence/timelapse of images (`rectype = imgseq`), a single video (`rectype = vid`), or multiple sessions of videos (`rectype = vidseq`). The "vidseq" recording type will record multiple videos with the same recording settings but wait after each finished recording for user input to continue with the next recording or exit. Each new video will be treated as a new "session" and have a corresponding session number in its filename (e.g. "S01", "S02" etc). The benefit of this recording option is that it is even quicker to record multiple videos one after the other with the same parameters and to have a simple automatic filenaming system that keeps those videos together.


## Position the camera and draw the region of interest

pirecorder comes with a `stream` module that displays an interactive live video stream on the desktop to help position the raspberry pi camera and objects in the camera view, draw a region of interest to be used for recordings, and zoom in on part of the video. It records the clicks and movements of the mouse and responds to the following keypresses:

- `f`-key: Display the stream fullscreen/display the stream in a window
- `c`-key: Display/hide a diagonal cross across the screen
- `s`-key: Save the coordinates of the rectangular area when drawn
- `e`-key: Erase the rectangular area when drawn
- `z`-key: Show a zoomed-in section of the video inside the rectangular area in maximum resolution
- `n`-key: Refresh the zoom-in image
- `o`-key: If the potential overlay image should be shown or not
- `[`- and `]`-keys: Decrease or increase the relative opacity of the potential overlay image with 5%
- `esc`-key: Exit the the zoom window as well as the calibrate function completely

The `stream` module can be run independently:

In [ ]:
pirecorder.Stream()

as part of a `PiRecorder` instance:

In [ ]:
rec.stream()

and directly from the command line:

In [ ]:
stream

If using the `stream` module to store the region of interest, the configfile needs to be added as a parameter, e.g. `stream --configfile pirecorder.conf`

To show that it stored the roi coordinates you can call the roi configuration directly

In [ ]:
rec.config.cus.roi

Clicking and drawing the mouse will draw a rectangular area. This can be used directly to store the coordinates of the region of interest that should be used for recordings, i.e. to only record the region inside the rectangular area.

Simply draw and redraw the rectangular area until you are happy and press the `s`-key. Now the coordinates of the region of interest will be displayed and, if running the `stream` functionality from a `PiRecorder` instance, stored automatically in the configuration file (e.g. `rec.config.cus.roi`). If you stored the region of interest accidentally or want to remove the drawn rectangle simple enter the `e`-key. To exit press the `esc`-key.


## Settings for image recording

To set the resolution for images you can use the `imgdims` parameter, which defaults to the maximum resolution for the v1.5 camera model (2592, 1944). The v2 model has a max resolution of 3280 x 2464 pixels, and the hq camera 4056 x 3040 pixels. The `imgquality` parameter specifies the quality that the jpeg encoder should attempt to maintain. Use values between 1 and 100, where higher values are higher quality. Playing with this setting can help to considerably reduce the file size of your recordings while keeping the same quality.

To control your image sequences you can set three parameters: `imgnr`, `imgtime`, and `imgwait`. PiRecorder will use the minimum of `imgnr` and the nr of images based on `imgwait` and `imgtime`. When the value provided for imgwait is too low relative to the provided shutterspeed it will be automatically set to the minimum value of 0.45s. With a fast enough shutterspeed it is possible to record multiple images per second, but depends on the model of raspberry pi you use. Also, when a delay is provided that is less than ~x5 the shutterspeed, the camera processing time will take more time than the provided imgwait parameter and so images are taken immediately one after the other. To take a sequence of images at the exact right delay interval the imgwait parameter should be at least 5x the shutterspeed (e.g. shutterspeed of 400ms needs imgwait of 2s.

For example, to record a sequence of 10 images at very high resolution at 1 image a minute:

In [ ]:
rec.settings(imgnr = 10, imgtime = 15, imgwait = 60, imgquality = 90, imgdims = (3280, 2464))

## Settings for video recording

For recording video you can set the `vidduration` and `viddelay` to get the right recording duration. The viddelay is extra recording time in seconds that will be added to vidduration. Its use is to add a standard amount of time to the video that can be easily cropped or skipped, such as for tracking, but still provides useful information.

To set the resolution for video recording use the `viddims` parameter. Note that the maximum video resolution for all currently existing raspberry pi camera's ("v1","v2" and "hq") is 1080p, but that it is also possible to record in a different format as long as the total number of pixels does not exceed that, such as 1640 x 1232. To set the framerate of the video use the `vidfps` parameter. With smaller resolutions higher framerates are possible.

The `vidquality` parameter specifies the quality that the h264 encoder should attempt to maintain. Use values between 10 and 40, where 10 is extremely high quality, and 40 is extremely low.

For example, to take a single video for 10 minutes with 20s extra time, with a 1640x1232 resolution at 24fps, with a relatively low quality and thus file size:

In [ ]:
rec.set_config(rectype = "vid", vidduration = 600, viddelay = 20, vidquality = 30, viddims = (1640, 1232), vidfps = 24)

## Camera settings

It is possible to set a large number of camera settings with pirecorder: `rotation`, `gains`, `brightness`, `contrast`, `saturation`, `iso`, `sharpness`, `compensation`, and `shutterspeed`. Besides setting these manually as explained above, there are also automatic and interactive modes.

By default, PiRecorder automatically sets the shutterspeed and white balance dynamically for and during each recording (`automode = True`). However, to get consistent recordings, it may be preferable to set the shutterspeed and white balance at a fixed value.

To get the optimal shutterspeed and white balance for the current conditions automatically you can use the `autoconfig` function, which will directly update the configuration file. This function will use the framerate provided in the configuration file for calibration, so make sure that is set properly. Then to run autoconfig, simply enter:

In [ ]:
rec.autoconfig()

## Change the camera settings interactively

pirecorder also comes with a very handy interactive tool (`camconfig`) that enables you to set the camera settings dynamically. `camconfig` opens a live video stream and a separate window with a trackbar for each of the camera settings. You can slide your parameters of interest between the possible values and see live how the resulting recording will look like. To run camconfig and store the values automatically in your configuration file, use the function linked to your PiRecorder instance:

In [ ]:
rec.camconfig()

You can exit the stream without saving with the `esc`-key and with saving with the `s`-key.

An important setting is the `automatic` mode. By default this is set to `True` such that it automatically gets the optimal shutterspeed and white balance (blue and red gains), which is visible by the respective trackbars sliding automatically to their optimal values. When you are relatively happy with these values it is a good time to use the non-automatic mode as then you are able to further tweak these values to your wishes.

`camconfig` will use the framerate as provided in the configuration file but can also use a custom framerate (e.g. `camconfig(fps=10)`), which will influence the range of shutterspeeds possible (see above). It is also possible to use camconfig stand alone without a PiRecorder instance. Simply import pirecorder and run the function:

In [ ]:
import pirecorder
pirecorder.Camconfig()

Then when the `s`-key is pressed it will output a dictionary of all values.

## Recording

To start your recordings, first import pirecorder, start your instance with the right configuration file, and use the `record` function:

In [ ]:
rec.record()

This will use all recording and camera settings as detailed in your configuration file. It is possible to run recordings immideatealy without any configuration. Then it will just use the default (`configfile = "pirecorder.conf"`) and use the automatic mode to dynamically set the right shutterspeed and white balance. You may wish to set the `rectype` and related parameters though to get the recording that you want.

It is also possible to run recordings straight from the terminal without requiring any further user input using the `record` command, which makes it very easy to run controlled recordings without requiring any user input. It will use the custom settings as provided in the configuration file, which you can change with the `--configfile` parameter:

In [ ]:
record --configfile "custom.conf"

## Recording different rectypes
With PiRecorder it is possible to record either A) a single image, b) a sequence of images, C) a single video, or D) a session of video recordings. Here are some examples to run these different types of recordings

In [ ]:
rec.settings(label = "testsingleimg", rectype = "img", imgdims = (1640,1232))

In [ ]:
rec.settings(label = "testimgseq", rectype = "imgseq", imgdims = (1640,1232), \
              imgwait = 3, imgnr = 10, imgtime = 20)

In [ ]:
rec.settings(label = "testvid", rectype = "vid", saturation = 50, vidduration = 7)

In [ ]:
rec.settings(label = "sessions", rectype = "vidseq", vidduration = 60)

To automatically save all images of a single image sequence in a folder set the `subdirs` parameter to `True`:

In [ ]:
rec.settings(subdirs = True)